**Objectif global :**

Ce code combine des techniques d'analyse d'images et d'extraction de texte pour identifier des tableaux dans un fichier PDF, extraire ces tableaux sous forme d'images, et convertir leur contenu en texte structuré.



In [ ]:
!pip install opencv-python-headless
!pip install pillow
!pip install pandas
!pip install pytesseract
!pip install scikit-learn
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-fra
!pip install ultralyticsplus==0.0.28 ultralytics==8.0.43

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (14.2 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123633 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [ ]:
!pip install pdf2image
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (944 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123684 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import cv2
import os
import pytesseract
from pytesseract import Output
from ultralyticsplus import YOLO
from pdf2image import convert_from_path
import pandas as pd

**Fonction 1 : Initialisation du modèle YOLO**

In [ ]:
def initialize_model():
    model = YOLO('foduucom/table-detection-and-extraction')
    model.overrides['conf'] = 0.25  # Seuil de confiance
    model.overrides['iou'] = 0.45  # Seuil IoU
    model.overrides['agnostic_nms'] = False
    model.overrides['max_det'] = 1000
    return model

**Fonction 2 : Extraction des tables sous forme d'images**

Conversion des pages PDF en images, détection et découpe des tableaux, stockage des images dans un dossier

In [ ]:
def extract_tables_from_pdf(pdf_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    model = initialize_model()
    table_image_paths = []

    pages = convert_from_path(pdf_path)
    print(f"{len(pages)} pages trouvées dans le PDF.")

    for page_number, page in enumerate(pages, start=1):
        page_image_path = os.path.join(output_dir, f'page_{page_number}.jpg')
        page.save(page_image_path, 'JPEG')

        image = cv2.imread(page_image_path)
        results = model.predict(page_image_path)

        for idx, box in enumerate(results[0].boxes, start=1):
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cropped_image = image[y1:y2, x1:x2]

            table_image_path = os.path.join(output_dir, f'table_page{page_number}_{idx}.jpg')
            cv2.imwrite(table_image_path, cropped_image)
            table_image_paths.append(table_image_path)
            print(f"Tableau enregistré : {table_image_path}")

    return table_image_paths

**Fonction 3 : Extraction du texte d'une image de table**

Analyse des images pour extraire et organiser le texte en conservant la mise en page.

In [ ]:
def extract_text_from_table_image(image_path):
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (int(img.shape[1] + (img.shape[1] * 0.1)),
                                   int(img.shape[0] + (img.shape[0] * 0.25))),
                             interpolation=cv2.INTER_AREA)
    img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)

    custom_config = r'-l eng --oem 3 --psm 6 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-:.$%./@& *"'
    d = pytesseract.image_to_data(img_rgb, config=custom_config, output_type=Output.DICT)
    df = pd.DataFrame(d)

    df1 = df[(df.conf != '-1') & (df.text != ' ') & (df.text != '')]

    sorted_blocks = df1.groupby('block_num').first().sort_values('top').index.tolist()
    extracted_text = ""

    for block in sorted_blocks:
        curr = df1[df1['block_num'] == block]
        char_w = (curr.width / curr.text.str.len()).mean()
        prev_par, prev_line, prev_left = 0, 0, 0
        text = ''

        for _, ln in curr.iterrows():
            if prev_par != ln['par_num']:
                text += '\n'
                prev_par = ln['par_num']
                prev_line = ln['line_num']
                prev_left = 0
            elif prev_line != ln['line_num']:
                text += '\n'
                prev_line = ln['line_num']
                prev_left = 0

            added = 0
            if ln['left'] / char_w > prev_left + 1:
                added = int((ln['left']) / char_w) - prev_left
                text += ' ' * added
            text += ln['text'] + ' '
            prev_left += len(ln['text']) + added + 1
        extracted_text += text + '\n'

    print(f"Contenu extrait de l'image {image_path} :\n{extracted_text}")
    print('-----------------------------------------------------------------------------------------')
    return extracted_text

**Script principal**

In [ ]:
def main(pdf_path, output_dir):
    print("Début du traitement...")
    table_images = extract_tables_from_pdf(pdf_path, output_dir)
    print(f"Nombre de tables extraites : {len(table_images)}")

    for image_path in table_images:
        extract_text_from_table_image(image_path)

**Exemple d'utilisation**

In [ ]:
pdf_path = "/content/sfcr_covea_2022.PDF"  # Remplacez par le chemin de votre PDF
output_dir = "tables_extracted"
os.makedirs(output_dir, exist_ok=True)

print("Début du traitement...")
table_images = extract_tables_from_pdf(pdf_path, output_dir) #Extraction des tables sous forme d'images dans le pdf
print(f"Nombre de tables extraites : {len(table_images)}")

Début du traitement...


/usr/local/lib/python3.10/dist-packages/ultralytics/nn/tasks.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


98 pages trouvées dans le PDF.


Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 /content/tables_extracted/page_1.jpg: 640x480 1 bordered, 879.6ms
Speed: 2.7ms preprocess, 879.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)



Tableau enregistré : tables_extracted/table_page1_1.jpg


...

Tableau enregistré : tables_extracted/table_page96_1.jpg


image 1/1 /content/tables_extracted/page_97.jpg: 480x640 1 borderless, 454.8ms
Speed: 1.0ms preprocess, 454.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



Tableau enregistré : tables_extracted/table_page97_1.jpg


image 1/1 /content/tables_extracted/page_98.jpg: 640x480 1 borderless, 450.7ms
Speed: 1.1ms preprocess, 450.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Tableau enregistré : tables_extracted/table_page98_1.jpg
Nombre de tables extraites : 59


In [ ]:
# Transformation des images en tables
for image_path in table_images:
        extract_text_from_table_image(image_path)

Contenu extrait de l'image tables_extracted/table_page1_1.jpg :

-----------------------------------------------------------------------------------------
Contenu extrait de l'image tables_extracted/table_page2_1.jpg :

AL. ACtIVILG eee  cccccccscsescseseseccscscsesssesesesssesavassssesesesssavesasssasacsesesasesasasasaeseseseuasacssassssesesasscacssssataeersasee G 
A2. RESultat de SOUSCTIPTION ..........csecesecseesesestsessesesesesesnseeseesseseststsnsesseseetsesteseasesasetstetstesetesetetseeeees LO 
A3. Resultat des INVEStISSEMENES  0.0... cccccsesesseeteessscscseseseseseseescscscssscsesesseetscscsssssesestsasssssssssasseseses QU 
A4. Resultat d@S AULFES ACTIVITES oo... ccc ccccseseescseecseseeseseescsesscsesecsesecsssessssecsestsessesesssesatsesstsecsseeases 2 

-----------------------------------------------------------------------------------------
Contenu extrait de l'image tables_extracted/table_page5_1.jpg :

            oe                            :                    er  